#Import statements and global variables assignement

In [1]:
import cv2
import numpy as np
import random,os,subprocess
from moviepy.editor import VideoFileClip, concatenate_videoclips

In [2]:
ideas_file='file.txt'
with open(ideas_file,'r') as file:
  ideas=file.readlines()
c=0
for i in range(len(ideas)):
  ideas[i]=ideas[i][0:len(ideas[i])-1]
  c+=len(ideas[i])
print(ideas)
print(c)

['Fondée en 1933,', 'contenant 260 milliards de barils', 'devant les autres société']
73


In [3]:
image_paths = ["image1.jpeg", "image2.jpg", "image3.jpg", "image4.jpeg"]
fps=24
total_duration = 0.5*c+0.5
output_path = "video.mp4"
final_video="final.mp4"
print(total_duration,' in seconds')
print(total_duration/60,' in minutes')

line_percentage=[0 for i in ideas]
for i in range(len(ideas)) :
  line_percentage[i]=(total_duration*len(ideas[i])/c)+0.5

print(line_percentage)

37.0  in seconds
0.6166666666666667  in minutes
[8.102739726027398, 17.226027397260275, 13.17123287671233]


#Animations

In [4]:
def zoom_in(image_path, output_path, duration=3, fps=30, max_scale=1.15):
    # Load the image
    image = cv2.imread(image_path)
    height, width, _ = image.shape

    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    # Calculate the scale increment per frame
    scale_increment = (max_scale - 1.0) / (duration * fps)

    # Calculate the center of the image
    center_x, center_y = width // 2, height // 2

    # Create the zoom-out frames
    current_scale = 1.0
    for i in range(int(duration * fps)):
        # Calculate the current scale for this frame
        current_scale += scale_increment

        # Calculate the translation to keep the center fixed
        translation_x = center_x * (1 - current_scale)
        translation_y = center_y * (1 - current_scale)

        # Create a scaling and translation matrix to zoom out the image from the center
        transformation_matrix = np.float32([[current_scale, 0, translation_x], [0, current_scale, translation_y]])

        # Apply the transformation to the image
        zoomed_in_image = cv2.warpAffine(image, transformation_matrix, (width, height))

        # Write the frame to the video
        out.write(zoomed_in_image)

    # Release the video writer
    out.release()


def zoom_out(image_path, output_path, duration=3, fps=30, max_scale=1.15):
    # Load the image
    image = cv2.imread(image_path)
    height, width, _ = image.shape

    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    # Calculate the scale decrement per frame
    scale_decrement = (max_scale - 1.0) / (duration * fps)

    # Calculate the center of the image
    center_x, center_y = width // 2, height // 2

    # Create the zoom-in frames
    current_scale = max_scale
    for i in range(int(duration * fps)):
        # Calculate the current scale for this frame
        current_scale -= scale_decrement

        # Calculate the translation to keep the center fixed
        translation_x = center_x * (1 - current_scale)
        translation_y = center_y * (1 - current_scale)

        # Create a scaling and translation matrix to zoom in the image from the center
        transformation_matrix = np.float32([[current_scale, 0, translation_x], [0, current_scale, translation_y]])

        # Apply the transformation to the image
        zoomed_out_image = cv2.warpAffine(image, transformation_matrix, (width, height))

        # Write the frame to the video
        out.write(zoomed_out_image)

    # Release the video writer
    out.release()

def translate_left(image_path, output_path, duration=3, fps=30):
    # Load the image
    image = cv2.imread(image_path)
    height, width, _ = image.shape

    # Calculate the width of the visible portion (90% of the image width)
    visible_width = int(width * 0.9)

    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (visible_width, height))

    # Calculate the total translation distance for the entire duration
    total_translation_distance = width - visible_width

    # Calculate the translation distance per second
    translation_per_second = total_translation_distance / duration

    # Create the translated frames
    for i in range(int(duration * fps)):
        # Calculate the translation distance for this frame
        translation_distance = int(i / fps * translation_per_second)

        # Create a translation matrix to shift the image to the left
        translation_matrix = np.float32([[1, 0, -translation_distance], [0, 1, 0]])

        # Apply the translation to the image
        translated_image = cv2.warpAffine(image, translation_matrix, (visible_width, height))

        # Write the frame to the video
        out.write(translated_image)

    # Release everything if job is finished
    out.release()


def translate_right(image_path, output_path, duration=3, fps=30):
    # Load the image
    image = cv2.imread(image_path)
    height, width, _ = image.shape

    # Calculate the width of the visible portion (90% of the image width)
    visible_width = int(width * 0.9)

    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (visible_width, height))

    # Calculate the total translation distance for the entire duration
    total_translation_distance = width - visible_width

    # Calculate the translation distance per second
    translation_per_second = total_translation_distance / duration

    # Create the translated frames
    frames = []
    for i in range(int(duration * fps)):
        # Calculate the translation distance for this frame
        translation_distance = int(i / fps * translation_per_second)

        # Create a translation matrix to shift the image to the left
        translation_matrix = np.float32([[1, 0, -translation_distance], [0, 1, 0]])

        # Apply the translation to the image
        translated_image = cv2.warpAffine(image, translation_matrix, (visible_width, height))

        # Append the frame to the frames list
        frames.append(translated_image)

    # Append the frames in reverse order to the output video to get the inverse effect
    for frame in reversed(frames):
        out.write(frame)

    # Release everything if job is finished
    out.release()

#video creation phase 1:

In [5]:
# Function to concatenate video clips into a single video using moviepy
def concatenate_videos(video_paths, output_path, target_resolution=(1280,720)):
    # Create VideoFileClip objects for each video and resize to the target resolution
    video_clips = [VideoFileClip(video_path).resize(target_resolution) for video_path in video_paths]

    # Concatenate the video clips
    final_clip = concatenate_videoclips(video_clips, method="compose")

    # Write the final video to the output path
    final_clip.write_videofile(output_path, codec="libx264", audio_codec="aac")

    # Close the video clips
    for video_clip in video_clips:
        video_clip.close()

# Function to create a video with transitions between images
def create_transition_video(image_paths, output_path, total_duration, fps=30, animations=None):
    if not animations:
        animations = [zoom_in, translate_left, zoom_out, translate_right]

    # Create temporary video animations for each image
    video_paths = []
    i=0
    for image_path in image_paths:
      video_path=f'temp_video_for_image_n_{i}.mp4'
      create_image_animation=random.choice(animations)
      create_image_animation(image_path, video_path, total_duration / len(image_paths), fps)
      video_paths.append(video_path)
      i+=1

    # Concatenate the temporary videos to create the final video
    concatenate_videos(video_paths, output_path)

    # Remove temporary video files
    for video_path in video_paths:
        os.remove(video_path)

In [6]:
create_transition_video(image_paths, output_path, total_duration, fps)

Moviepy - Building video video.mp4.
Moviepy - Writing video video.mp4



Moviepy - Done !
Moviepy - video ready video.mp4


#Video creation phase 2:

##CV2

###Option 1

In [7]:
import cv2
import numpy as np

# Load the video file
cap = cv2.VideoCapture(output_path)
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))


# Calculate the start and end frames for each subtitle
start_frames = [int(sum(line_percentage[:i]) * fps) for i in range(len(line_percentage))]
end_frames = [int(sum(line_percentage[:i+1]) * fps) for i in range(len(line_percentage))]

# Set up the video writer
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(final_video, fourcc, fps, (int(cap.get(3)), int(cap.get(4))))

# Add the subtitles to the video
frame_num = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Check if we need to add a subtitle to this frame
    for i, (start, end) in enumerate(zip(start_frames, end_frames)):
        if start <= frame_num < end:
            # Add the subtitle to the frame
            font = cv2.FONT_HERSHEY_SIMPLEX
            text = ideas[i]
            textsize = cv2.getTextSize(text, font, 1, 2)[0]
            textX = (frame.shape[1] - textsize[0]) // 2
            textY = frame.shape[0] - 50
            cv2.putText(frame, text, (textX, textY), font, 1, (255, 255, 255), 2)

    # Write the frame to the output video
    out.write(frame)
    frame_num += 1

# Release resources
cap.release()
out.release()


###Option 2

In [8]:
import cv2
import numpy as np
import random

# Load the video file
cap = cv2.VideoCapture(output_path)
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# Calculate the start and end frames for each subtitle
start_frames = [int(sum(line_percentage[:i]) * fps) for i in range(len(line_percentage))]
end_frames = [int(sum(line_percentage[:i+1]) * fps) for i in range(len(line_percentage))]

# Set up the video writer
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(final_video, fourcc, fps, (int(cap.get(3)), int(cap.get(4))))

# Define the subtitle animations
animations = ['left', 'right', 'top', 'bottom', 'fade']

def apply_animation(frame, text, animation, progress):
    font = cv2.FONT_HERSHEY_SIMPLEX
    textsize = cv2.getTextSize(text, font, 1, 2)[0]
    text_width, text_height = textsize
    x_center = (frame.shape[1] - text_width) // 2
    y_center = frame.shape[0] - 50

    if animation == 'left':
        x_pos = int(x_center * progress)
        y_pos = y_center
    elif animation == 'right':
        x_pos = int(x_center + (frame.shape[1] - x_center) * (1 - progress))
        y_pos = y_center
    elif animation == 'top':
        x_pos = x_center
        y_pos = int(y_center * progress)
    elif animation == 'bottom':
        x_pos = x_center
        y_pos = int(y_center + (frame.shape[0] - y_center) * (1 - progress))
    elif animation == 'fade':
        x_pos = x_center
        y_pos = y_center

    # Add a black background to the subtitle
    bg_thickness = 3
    bg_color = (0, 0, 0)
    top_left = (x_pos - bg_thickness, y_pos + bg_thickness)
    bottom_right = (x_pos + text_width + bg_thickness, y_pos - text_height - bg_thickness)
    cv2.rectangle(frame, top_left, bottom_right, bg_color, -1)

    # Add the subtitle to the frame
    font_color = (241, 213, 0)
    cv2.putText(frame, text, (x_pos, y_pos), font, 1, font_color, 2)

    # Apply the fade animation if necessary
    if animation == 'fade':
        alpha = progress
        beta = 1 - alpha
        frame[:] = cv2.addWeighted(frame, alpha, np.zeros_like(frame), beta, 0)

# Add the subtitles to the video
frame_num = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Check if we need to add a subtitle to this frame
    for i, (start, end) in enumerate(zip(start_frames, end_frames)):
        if start <= frame_num < end:
            # Calculate the progress of the animation
            progress = (frame_num - start) / (end - start)

            # Apply a random animation to the subtitle
            animation = random.choice(animations)
            apply_animation(frame, ideas[i], animation, progress)

    # Write the frame to the output video
    out.write(frame)
    frame_num += 1

# Release resources
cap.release()
out.release()


##Moviepy

###Option 1

In [9]:
from moviepy.editor import VideoFileClip, TextClip
from moviepy.video.tools.subtitles import SubtitlesClip

# Load the video file
video_clip = VideoFileClip(output_path)

# Create subtitle clips
subtitles = []
start_time = 0
for idea, duration in zip(ideas, line_percentage):
    end_time = start_time + duration
    subtitle = TextClip(idea, fontsize=24, color='white', bg_color='black', size=video_clip.size)
    subtitles.append(((start_time, end_time), subtitle))
    start_time = end_time

# Create the subtitle composition
subtitles_composition = SubtitlesClip(subtitles)

# Overlay subtitles on the video
video_with_subtitles = CompositeVideoClip([video_clip, subtitles_composition.set_duration(video_clip.duration)])

# Export the final video
video_with_subtitles.write_videofile(final_video, codec='libx264')


OSError: ignored

###Option 2

In [10]:
import random
from moviepy.editor import VideoFileClip, TextClip
from moviepy.video.tools.subtitles import SubtitlesClip

# Load the video file
video_clip = VideoFileClip(output_path)

# Function to generate a random animation
def random_animation(txt_clip, width, duration):
    animation_list = [
        txt_clip.set_position(('left', 'bottom')).set_duration(duration).margin(left=10, right=10, bottom=10),
        txt_clip.set_position(('right', 'top')).set_duration(duration).margin(right=10, top=10),
        txt_clip.set_position(('center', 'bottom')).set_duration(duration).margin(bottom=10),
        txt_clip.set_position(('center', 'top')).set_duration(duration).margin(top=10),
        txt_clip.set_position(('left', 'center')).set_duration(duration).margin(left=10),
        txt_clip.set_position(('right', 'center')).set_duration(duration).margin(right=10)
    ]
    return random.choice(animation_list)

# Create subtitle clips with animations
subtitles = []
start_time = 0
for idea, duration in zip(ideas, line_percentage):
    end_time = start_time + duration

    # Create text clip with specified style
    txt_clip = TextClip(idea, fontsize=24, color='#00D5F1', bg_color='black', size=video_clip.size)

    # Apply a random animation to the text clip
    animated_clip = random_animation(txt_clip, video_clip.w, duration)

    subtitles.append(((start_time, end_time), animated_clip))
    start_time = end_time

# Create the subtitle composition
subtitles_composition = SubtitlesClip(subtitles)

# Overlay subtitles on the video
video_with_subtitles = CompositeVideoClip([video_clip, subtitles_composition.set_duration(video_clip.duration)])

# Export the final video
video_with_subtitles.write_videofile(final_video, codec='libx264')


OSError: ignored